# Setup and Imports

In [ ]:
!pip install -U -q timm transformers trl peft

In [ ]:
import io
import os
import zipfile

import torch
from datasets import DatasetDict, load_dataset, Audio
from huggingface_hub import hf_hub_download, list_repo_files
from PIL import Image
from transformers import AutoModelForImageTextToText, AutoProcessor, Gemma3nForConditionalGeneration
from peft import LoraConfig

from trl import (
    ModelConfig,
    ScriptArguments,
    SFTConfig,
    SFTTrainer,
    TrlParser,
    get_kbit_device_map,
    get_quantization_config,
)

# Dataset


In [ ]:
dataset = load_dataset("AdrienB134/Emilia-dataset-french-split", split="fr")
dataset = dataset.select(range(1000))
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
def format_intersection_data(samples: dict) -> dict[str, list]:
    """Format intersection dataset to match expected message format"""
    formatted_samples = {"messages": []}
    for idx in range(len(samples["audio"])):
        audio = samples["audio"][idx]["array"]
        label = str(samples["text"][idx])

        message = [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "You are an assistant that transcribes speech accurately.",
                    }
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "audio", "audio": audio},
                    {"type": "text", "text": "Please transcribe this audio."}
                ]
            },
            {
                "role": "assistant",
                "content":[{"type": "text", "text": label}]
            }
        ]
        formatted_samples["messages"].append(message)
    return formatted_samples

In [ ]:
dataset = dataset.map(format_intersection_data, batched=True, batch_size=4, num_proc=4)

In [ ]:
model = Gemma3nForConditionalGeneration.from_pretrained(
    "google/gemma-3n-E2B-it", trust_remote_code=True, torch_dtype=torch.bfloat16,
)
processor = AutoProcessor.from_pretrained(
    "google/gemma-3n-E2B-it", trust_remote_code=True,
)
processor.tokenizer.padding_side = "right"

In [ ]:
def collate_fn(examples):
        texts = []
        audios = []

        for example in examples:
            # Apply chat template to get text
            text = processor.apply_chat_template(
                example["messages"], tokenize=False, add_generation_prompt=False
            ).strip()
            texts.append(text)

            # Extract audios
            audios.append(example["audio"]["array"])

        # Tokenize the texts and process the images
        batch = processor(
            text=texts, audio=audios, return_tensors="pt", padding=True
        )

        # The labels are the input_ids, and we mask the padding tokens in the loss computation
        labels = batch["input_ids"].clone()

        # Use Gemma3n specific token masking
        labels[labels == processor.tokenizer.pad_token_id] = -100
        if hasattr(processor.tokenizer, 'image_token_id'):
            labels[labels == processor.tokenizer.image_token_id] = -100
        if hasattr(processor.tokenizer, 'audio_token_id'):
            labels[labels == processor.tokenizer.audio_token_id] = -100
        if hasattr(processor.tokenizer, 'boi_token_id'):
            labels[labels == processor.tokenizer.boi_token_id] = -100
        if hasattr(processor.tokenizer, 'eoi_token_id'):
            labels[labels == processor.tokenizer.eoi_token_id] = -100


        batch["labels"] = labels
        return batch

# Training


In [ ]:
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0.00,
    bias="none",
    use_rslora=False,
    use_dora=False,
    modules_to_save=None,
)

In [ ]:
training_args = SFTConfig(
    output_dir="gemma-3n-E2B-it-trl-sft",
    eval_strategy='no',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    learning_rate=1e-05,
    num_train_epochs=1.0,
    logging_steps=10,
    save_steps=100,
    bf16=True,
    report_to=["wandb"],
    run_name='gemma-3n-E2B-it-trl-sft',
    dataset_kwargs={'skip_prepare_dataset': True},
    remove_unused_columns=False,
    max_seq_length=None,
)

In [ ]:
split_ds = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_ds["train"]
val_dataset = split_ds["test"]

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=val_dataset if training_args.eval_strategy != "no" else None,
    processing_class=processor.tokenizer,
    peft_config=peft_config,
)

In [ ]:
trainer.train()